## Import Libraries

In [24]:
import time
import rospy
import numpy as np
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from dual_quaternion import Quaternion
from dual_quaternion import DualQuaternion
from casadi import Function

## Working with Symbolic Variables

In [25]:
# Defining Mobile Robo Frame
theta_1 = ca.MX.sym('theta_1', 1, 1)
n_1 = ca.MX([0.0, 0.0, 1.0])
q_1 = ca.vertcat(ca.cos(theta_1/2), ca.sin(theta_1/2)@n_1)
tx_1 = ca.MX.sym("x_1", 1, 1)
ty_1 = ca.MX.sym("y_1", 1, 1)
t_1 = ca.vertcat(0.0, tx_1, ty_1, 0.0)

# Time Derivaties
theta_1p = ca.MX.sym('theta_1p', 1, 1)
tx_1p = ca.MX.sym("x_1p", 1, 1)
ty_1p = ca.MX.sym("y_1p", 1, 1)

aux_symbolic_total = ca.vertcat(tx_1, ty_1, theta_1)
symbolic_actions = ca.vertcat(tx_1, ty_1, theta_1)
dual_1_data = ca.vertcat(tx_1, ty_1, theta_1)

control = ca.vertcat(tx_1p, ty_1p, theta_1p)

In [26]:
# Defining Desired Frame
theta_1d = ca.MX.sym('theta_1d', 1, 1)
n_1d = ca.MX([0.0, 0.0, 1.0])
q_1d = ca.vertcat(ca.cos(theta_1d/2), ca.sin(theta_1d/2)@n_1d)
tx_1d = ca.MX.sym("x_1d", 1, 1)
ty_1d = ca.MX.sym("y_1d", 1, 1)
t_1d = ca.vertcat(0.0, tx_1d, ty_1d, 0.0)

# Time Derivaties
theta_1pd = ca.MX.sym('theta_1pd', 1, 1)
tx_1pd = ca.MX.sym("x_1pd", 1, 1)
ty_1pd = ca.MX.sym("y_1pd", 1, 1)

dual_1d_data = ca.vertcat(tx_1d, ty_1d, theta_1d)

# Defining the desired Velocity
vx_1d = ca.MX.sym("vx_1d", 1, 1)
vy_1d = ca.MX.sym("vy_1d", 1, 1)

wz_1d = ca.MX.sym("wz_1d", 1, 1)

Vd = ca.vertcat(0.0, vx_1d, vy_1d, 0.0)
Wd = ca.vertcat(0.0, 0.0, 0.0, wz_1d)

w_1d_data = ca.vertcat(vx_1d, vy_1d, wz_1d)

## Control Gains

In [27]:
# Defining the desired Velocity
kr1 = ca.MX.sym("kr1", 1, 1)
kr2 = ca.MX.sym("kr2", 1, 1)
kr3 = ca.MX.sym("kr3", 1, 1)

kd1 = ca.MX.sym("kd1", 1, 1)
kd2 = ca.MX.sym("kd2", 1, 1)
kd3 = ca.MX.sym("kd3", 1, 1)


Kr = ca.vertcat(0.0, kr1, kr2, kr3)
Kd = ca.vertcat(0.0, kd1, kd2, kd3)

k_data = ca.vertcat(kr1, kr2, kr3, kd1, kd2, kd3)

### Computing Sequential Transformations

In [28]:
Q1 = DualQuaternion.from_pose(quat = q_1, trans = t_1)
Q1.get_dual

Quaternion: (0.5*mac(horzcat(horzcat(0, (-x_1), (-y_1), 0)', horzcat(x_1, 0, 0, y_1)', horzcat(y_1, 0, 0, (-x_1))', horzcat(0, (-y_1), x_1, 0)')',vertcat(cos((theta_1/2)), ([0, 0, 1]*sin((theta_1/2)))),zeros(4x1)))

In [29]:
Q1d = DualQuaternion.from_pose(quat = q_1d, trans = t_1d)


In [30]:
W1d = DualQuaternion(q_real= Quaternion(q = Wd), q_dual= Quaternion(q= Vd))

In [31]:
Kp =  DualQuaternion(q_real= Quaternion(q = Kr), q_dual= Quaternion(q= Kd))

## Generating Function Using CasADi

In [32]:
Function_DualQuat_total = Function('DualQuat_total', [aux_symbolic_total], [Q1.get[:, 0]])
Function_DualQuat_total_trans = Function('DualQuat_total_trans', [aux_symbolic_total], [Q1.get_trans.get[:, 0]])
Function_DualQuat_total_quat = Function('DualQuat_total_quat', [aux_symbolic_total], [Q1.get_quat.get[:, 0]])

## Using Functions

In [33]:
input_t = np.array([-2, 5, np.pi/2])
dual = Function_DualQuat_total(input_t)
dual

DM([0.707107, 0, 0, 0.707107, 0, 1.06066, 2.47487, 0])

In [34]:
input_t = np.array([-2, 5, np.pi/2])
trans = Function_DualQuat_total_trans(input_t)
trans

DM([0, -2, 5, 0])

In [35]:
input_t = np.array([-2, 5, np.pi/2])
quat = Function_DualQuat_total_quat(input_t)
quat

DM([0.707107, 0, 0, 0.707107])

### Calculating Jacobians

In [36]:
J = ca.jacobian(Q1.get[:, 0], symbolic_actions)
J.shape

(8, 3)

In [37]:
Function_J = Function("Function_J", [aux_symbolic_total], [J])

In [38]:
input_t = np.array([5, 5, 0.55])
Jacobian = Function_J(input_t)
print(Jacobian)


[[00, 00, -0.135773], 
 [00, 00, 0], 
 [00, 00, 0], 
 [00, 00, 0.481213], 
 [-0, -0, 0], 
 [0.481213, 0.135773, 0.863598], 
 [-0.135773, 0.481213, -1.54247], 
 [0, -0, 0]]


## Computing Differential Kinematics

In [39]:
diff_kinematics = J @ control

Function_diff_kinematics = Function("Function_diff_kinematics", [aux_symbolic_total, control], [diff_kinematics])

In [40]:
input_t = np.array([-3.15, 0.0, 1])
input_t_p = np.array([0.36, 0.0, 0.0])
Xp = Function_diff_kinematics(input_t, input_t_p)
Xp

DM([0, 0, 0, 0, 0, 0.157965, -0.0862966, 0])

In [41]:
# Compute short path to the desired quaternion
q_error = ca.MX.sym('q_error', 8, 1)

# Define the function
f_error = ca.Function('f_error', [q_error], [ca.if_else(q_error[0, 0] >= 0, q_error, -q_error)])

In [42]:
aux = np.array([-2.0, 1.0, 0.0, -3, 5, -1, 8, 0])
print(aux)

aux = f_error(aux)
print(aux)

[-2.  1.  0. -3.  5. -1.  8.  0.]
[2, -1, 0, 3, -5, 1, -8, 0]


## Computing error

In [43]:
def dual_control(qd, wd, q, kp, qd_data, q_data, wd_data, kp_data):
    q_e = q * qd.conjugate()
    
    # Shortest p

    q_error = q_e.get[:, 0]

    q_error = f_error(q_error)

    q_e.error_path

    # Apply log mapping
    q_e_ln = q_e.ln_control()

    # Conjugate
    q_e_c = q_e.conjugate()

    # Control Law 
    U = -2*q_e_ln.vector_dot_product(kp) + q_e * wd * q_e_c

    U_control = Function('U_control', [qd_data, q_data, wd_data, kp_data], [U.get[:, 0]])
    return U_control

In [44]:
control_law = dual_control(Q1d, W1d, Q1, Kp, dual_1d_data, dual_1_data, w_1d_data, k_data)

In [45]:
dual_1 = np.array([2.0, 2.0, 0.0])
dual_1d = np.array([2.0, -2.0, np.pi/2])


In [46]:
qe = qe_function(dual_1, dual_1d)
qe

NameError: name 'qe_function' is not defined